In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
dataset_repository = "D:\manualCDmanagement\codes\Projects\VMs\skl algorithms\Logistic Regression/00_datasets\Loan.1"
dataset_name = "loan.csv"
dataset_file = os.path.join(dataset_repository, dataset_name)

df = pd.read_csv(dataset_file)

df['gender'], gender_uniques = pd.factorize(df.gender)
df['occupation'], occupation_uniques = pd.factorize(df['occupation'])
df['education_level'], educ_level_uniques = pd.factorize(df['education_level'])
df.marital_status, marital_status_uniques = pd.factorize(df.marital_status)
df.loan_status, loan_status_uniques = pd.factorize(df.loan_status)

df.head()

,age,gender,occupation,education_level,marital_status,income,credit_score,loan_status
0,32,0,0,0,0,85000,720,0
1,45,1,1,1,1,62000,680,0
2,28,0,2,2,1,25000,590,1
3,51,1,3,0,0,105000,780,0
4,36,0,4,0,0,75000,710,0


In [3]:
X = df.drop('loan_status', axis=1)
y = df.loan_status

scaler = StandardScaler()
X_scaled = scaler.fit(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)